In [1]:
import sys
import os
import pandas as pd
notebook_dir = os.getcwd()
sys.path.insert(0, os.path.abspath(os.path.join(notebook_dir, '..')))  # Add the project root directory to the path
import custom_io as cio
import env_reader
import matplotlib.pyplot as plt
import h5py
import numpy as np

In [2]:
save_fig = True

In [3]:
er = env_reader.read_env()
output_dir = er["OUTPUT_FOLDER"]

In [4]:
def map_window_type(win_type: str):
    """
    Convert the window type to a standard naming convention

    Args:
        win_type (str): _description_

    Returns:
        _type_: _description_
    """
    if "ca1" in win_type.lower():
        return "CA1"
    if "cx" in win_type.lower() or "ctx" in win_type.lower() or "nc" in win_type.lower():
        return "CTX"

In [5]:
fpath_assembled_traces = cio.open_file("Open assembled traces h5 file")

In [6]:
dict_traces = dict()
with h5py.File(fpath_assembled_traces, "r") as f:
    for uuid in f.keys():
        has_lfp = f[uuid].attrs['has_lfp']
        n_recordings = len(f[uuid].attrs["recording_break_points"][()])
        if n_recordings > 1:  # each recording should have its own lfp
            has_lfp = has_lfp.all()
        segment_type_break_points_lfp = None if not has_lfp else f[uuid].attrs['segment_type_break_points_lfp'][()] 
        dict_traces[uuid] = {
            'mouse_id': f[uuid].attrs['mouse_id'],
            'window_type': map_window_type(f[uuid].attrs['window_type']),
            'has_lfp': has_lfp,
            'lfp_t': f[uuid]['lfp_t'][()],
            'lfp_y': f[uuid]['lfp_y'][()],
            'lfp_mov_y': f[uuid]['lfp_mov_y'][()],
            'lv_speed': f[uuid]['lv_speed'][()],
            'lv_t_s': f[uuid]['lv_t_s'][()],
            'mean_fluo': f[uuid]['mean_fluo'][()],
            'segment_type_break_points': f[uuid].attrs['segment_type_break_points'],
            'segment_type_break_points_lfp': segment_type_break_points_lfp,
        }

In [7]:
uuids_list = list(dict_traces.keys())

In [8]:
def plot_fluo_lfp(uuid, t_before_sz_begin=60):
    has_lfp = dict_traces[uuid]['has_lfp']
    if not has_lfp:
        return
    i_begin_sz = dict_traces[uuid]['segment_type_break_points'][1]
    i_end_sz = dict_traces[uuid]['segment_type_break_points'][2]
    i_end_sz_lfp = dict_traces[uuid]['segment_type_break_points_lfp'][2]
    t_sz_begin = dict_traces[uuid]['lv_t_s'][i_begin_sz]
    # get 10 s before sz
    i_begin_plot =  dict_traces[uuid]['lv_t_s'].searchsorted(t_sz_begin - t_before_sz_begin)
    i_begin_plot_lfp = dict_traces[uuid]['lfp_t'].searchsorted(t_sz_begin - t_before_sz_begin)
    fluo_x = dict_traces[uuid]['lv_t_s'][i_begin_plot:i_end_sz]
    fluo_y = dict_traces[uuid]['mean_fluo'][i_begin_plot:i_end_sz]
    lfp_x = dict_traces[uuid]['lfp_t'][i_begin_plot_lfp:i_end_sz_lfp]
    lfp_y = dict_traces[uuid]['lfp_y'][i_begin_plot_lfp:i_end_sz_lfp]
    fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    axs[0].plot(fluo_x, fluo_y, linewidth=0.5)
    axs[1].plot(lfp_x, lfp_y, linewidth=0.5, color="red")
    plt.suptitle(f"Mouse {dict_traces[uuid]['mouse_id']}, window type {dict_traces[uuid]['window_type']}")
    plt.show()

In [37]:
dict_sz_begin_lfp_s = {
    "2251bba132cf45fa839d3214d1651392": 293.6,
    "7b9c17d8a1b0416daf65621680848b6a": 328.35,
    "9e75d7135137444492d104c461ddcaac": 328.25,
    "c7b29d28248e493eab02288b85e3adee": 328.25,
    "cd3c1e0e3c284a89891d2e4d9a7461f4": 315.3,
    "f5ccb81a34bb434482e2498bfdf88784": 325}

# the optical sz begin is already quantified. Optical sz end is not properly quantified, it was marking the appearance of SD waves (CA1) or a rough end (CTX)
dict_sz_begin_optical_s = {
    '2251bba132cf45fa839d3214d1651392': 328.3,
    '7b9c17d8a1b0416daf65621680848b6a': 328.24,
    '9e75d7135137444492d104c461ddcaac': 328.23,
    'c7b29d28248e493eab02288b85e3adee': 328.25,
    'cd3c1e0e3c284a89891d2e4d9a7461f4': 319.6,
    'f5ccb81a34bb434482e2498bfdf88784': 328
}


dict_sz_end_optical_s = {
    "2251bba132cf45fa839d3214d1651392": 360.5,
    "7b9c17d8a1b0416daf65621680848b6a": 347.5,
    "9e75d7135137444492d104c461ddcaac": 342.7,
    "c7b29d28248e493eab02288b85e3adee": 347.2,
    "cd3c1e0e3c284a89891d2e4d9a7461f4": 390,
    "f5ccb81a34bb434482e2498bfdf88784": 367.5
}

dict_sz_end_lfp_s = {
    "2251bba132cf45fa839d3214d1651392": 374,
    "7b9c17d8a1b0416daf65621680848b6a": 347.5,
    "9e75d7135137444492d104c461ddcaac": 343,
    "c7b29d28248e493eab02288b85e3adee": 347.5,
    "cd3c1e0e3c284a89891d2e4d9a7461f4": 396,
    "f5ccb81a34bb434482e2498bfdf88784": 367.7
}

In [ ]:
# overview plot
n_recordings = len(dict_sz_begin_lfp_s)
fig, axs = plt.subplots(2*n_recordings, 1,figsize=(15, 10*n_recordings))
i_plot = 0
t_offset = 10  # +- 5 s to plot
for uuid in dict_sz_begin_lfp_s.keys():
    i_begin_plot = dict_traces[uuid]['segment_type_break_points'][1]
    i_end_plot_lfp = dict_traces[uuid]['segment_type_break_points_lfp'][2]
    t_sz_end_optical = dict_sz_end_optical_s[uuid]
    t_sz_begin_optical = dict_sz_begin_optical_s[uuid]
    t_sz_begin_lfp = dict_sz_begin_lfp_s[uuid]
    t_sz_end_optical = dict_sz_end_optical_s[uuid]
    t_sz_end_lfp = dict_sz_end_lfp_s[uuid]
    i_end_plot = dict_traces[uuid]["lv_t_s"].searchsorted(t_sz_end_optical)

    t_begin_plot = min(t_sz_begin_optical, t_sz_begin_lfp) - t_offset
    t_end_plot = max(t_sz_end_optical, t_sz_end_lfp) + t_offset

    i_begin_plot = dict_traces[uuid]['lv_t_s'].searchsorted(t_begin_plot)
    i_end_plot = dict_traces[uuid]['lv_t_s'].searchsorted(t_end_plot)
    i_begin_plot_lfp = dict_traces[uuid]['lfp_t'].searchsorted(t_begin_plot)
    i_end_plot_lfp = dict_traces[uuid]['lfp_t'].searchsorted(t_end_plot)

    fluo_x = dict_traces[uuid]['lv_t_s'][i_begin_plot:i_end_plot]
    fluo_y = dict_traces[uuid]['mean_fluo'][i_begin_plot:i_end_plot]
    lfp_x = dict_traces[uuid]['lfp_t'][i_begin_plot_lfp:i_end_plot_lfp]
    lfp_y = dict_traces[uuid]['lfp_y'][i_begin_plot_lfp:i_end_plot_lfp]
    axs[i_plot].plot(fluo_x, fluo_y, linewidth=0.5, color="green")
    axs[i_plot].set_title(f"{uuid}: {dict_traces[uuid]['mouse_id']}, {dict_traces[uuid]['window_type']}")
    axs[i_plot].vlines([t_sz_begin_lfp, t_sz_end_lfp], ymin=np.min(fluo_y), ymax=np.max(fluo_y), color="blue")  # lfp seizure start, end
    axs[i_plot].vlines([t_sz_begin_optical, t_sz_end_optical], ymin=np.min(fluo_y), ymax=np.max(fluo_y), color="green")  # optical seizure start, end
    axs[i_plot].set_xlim([t_begin_plot, t_end_plot])
    i_plot += 1
    axs[i_plot].plot(lfp_x, lfp_y, linewidth=0.5, color="blue")
    axs[i_plot].vlines([t_sz_begin_lfp, t_sz_end_lfp], ymin=np.min(lfp_y), ymax=np.max(lfp_y), color="blue")  # lfp seizure start, end
    axs[i_plot].vlines([t_sz_begin_optical, t_sz_end_optical], ymin=np.min(lfp_y), ymax=np.max(lfp_y), color="green")  # optical seizure start, end
    axs[i_plot].set_xlim([t_begin_plot, t_end_plot])
    i_plot += 1
if save_fig:
    fpath_out = os.path.join(output_dir, "sz_begin_end_tmev.pdf")
    plt.savefig(fpath_out, format="pdf", bbox_inches="tight")
    print(f"Saved to {fpath_out}")
plt.show()

In [ ]:
# print the different times
for uuid in dict_sz_begin_lfp_s.keys():
    t_sz_begin_lfp = dict_sz_begin_lfp_s[uuid]
    t_sz_begin_optical = dict_sz_begin_optical_s[uuid]
    t_sz_end_lfp = dict_sz_end_lfp_s[uuid]
    t_sz_end_optical = dict_sz_end_optical_s[uuid]
    print(f"{uuid}:\noptical:\n\t{t_sz_begin_optical} - {t_sz_end_optical}\nlfp:\n\t{t_sz_begin_lfp} - {t_sz_end_lfp}\n")

## Show optical sz begin and end for all other recordings (i.e. those without good LFP)

In [12]:
uuids_bad_lfp = []
for uuid in dict_traces.keys():
    if uuid not in dict_sz_begin_lfp_s.keys():
        uuids_bad_lfp.append(uuid)

In [13]:
n_bad_recordings_with_lfp = 0
for uuid in uuids_bad_lfp:
    if dict_traces[uuid]['has_lfp']:
        n_bad_recordings_with_lfp += 1

In [14]:
dict_sz_begin_optical_bad_lfp_s = {
    '30bcfb76a771468eab5c2a0bb71038d7': 328.17,
    '39f7ef9f661041428bdd57a5b15c7176': 328.2,
    '4dea78a01bf5408092f498032d67d84e': 328.27,
    '4e2310d2dde845b0908519b7196080e8': 328.21,
    '54c31c3151944cfd86043932d3a19b9a': 338.83,
    '58dbee01eacf4b7385e0192c812233da': 328.14,
    '5cfb012d47f14303a40680d2b333336a': 328.15,
    '5ecdd9dc8f13440f9becae3cde5ab574': 328.21,
    '74473c5d22e04525acf53f5a5cb799f4': 328.15,
    '7753b03a2a554cccaab42f1c0458d742': 339.66,
    'a39ed3a880c54f798eff250911f1c92f': 328.23,
    'aa66ae0470a14eb08e9bcadedc34ef64': 328.17,
    'd158cd12ad77489a827dab1173a933f9': 329.09,
    'f0442bebcd1a4291a8d0559eb47df08e': 361.26,
    'f481149fa8694621be6116cb84ae2d3c': 351.03
}

dict_sz_end_optical_bad_lfp_s = {
    '30bcfb76a771468eab5c2a0bb71038d7': 388.81,
    '39f7ef9f661041428bdd57a5b15c7176': 379.72,
    '4dea78a01bf5408092f498032d67d84e': 349,
    '4e2310d2dde845b0908519b7196080e8': 359.39,
    '54c31c3151944cfd86043932d3a19b9a': 363,
    '58dbee01eacf4b7385e0192c812233da': 376.97,
    '5cfb012d47f14303a40680d2b333336a': 391,
    '5ecdd9dc8f13440f9becae3cde5ab574': 364.77,
    '74473c5d22e04525acf53f5a5cb799f4': 392.67,
    '7753b03a2a554cccaab42f1c0458d742': 352,
    'a39ed3a880c54f798eff250911f1c92f': 364.46,
    'aa66ae0470a14eb08e9bcadedc34ef64': 348.92,
    'd158cd12ad77489a827dab1173a933f9': 369.21,
    'f0442bebcd1a4291a8d0559eb47df08e': 361.46,
    'f481149fa8694621be6116cb84ae2d3c': 359
}

In [ ]:
n_recordings_bad_lfp = len(uuids_bad_lfp)
fig, axs = plt.subplots(n_recordings_bad_lfp + n_bad_recordings_with_lfp, 1,figsize=(15, 10*n_recordings_bad_lfp))
i_plot_bad_lfp = 0
t_offset_bad_lfp = 5  # +- 5 s to plot
for uuid in uuids_bad_lfp:
    t_sz_begin_optical = dict_sz_begin_optical_bad_lfp_s[uuid]
    t_sz_end_optical = dict_sz_end_optical_bad_lfp_s[uuid]
    t_begin_plot = t_sz_begin_optical - t_offset_bad_lfp
    t_end_plot = t_sz_end_optical + t_offset_bad_lfp
    has_lfp = dict_traces[uuid]['has_lfp']
    i_begin_plot = np.searchsorted(dict_traces[uuid]['lv_t_s'], t_begin_plot)
    i_end_plot = np.searchsorted(dict_traces[uuid]['lv_t_s'], t_end_plot)

    fluo_x = dict_traces[uuid]['lv_t_s'][i_begin_plot:i_end_plot]
    fluo_y = dict_traces[uuid]['mean_fluo'][i_begin_plot:i_end_plot]
    
    if has_lfp:
        i_begin_plot_lfp = np.searchsorted(dict_traces[uuid]['lfp_t'], t_begin_plot)
        i_end_plot_lfp = np.searchsorted(dict_traces[uuid]['lfp_t'], t_end_plot)
        lfp_x = dict_traces[uuid]['lfp_t'][i_begin_plot_lfp:i_end_plot_lfp]
        lfp_y = dict_traces[uuid]['lfp_y'][i_begin_plot_lfp:i_end_plot_lfp]
    axs[i_plot_bad_lfp].plot(fluo_x, fluo_y, linewidth=0.5, color="green")
    axs[i_plot_bad_lfp].set_title(f"{uuid}: {dict_traces[uuid]['mouse_id']}, {dict_traces[uuid]['window_type']}")
    axs[i_plot_bad_lfp].vlines([t_sz_begin_optical, t_sz_end_optical], ymin=np.min(fluo_y), ymax=np.max(fluo_y), color="green")  # optical seizure start, end
    axs[i_plot_bad_lfp].set_xlim([t_begin_plot, t_end_plot])
    i_plot_bad_lfp += 1
    if has_lfp:
        axs[i_plot_bad_lfp].plot(lfp_x, lfp_y, linewidth=0.5, color="blue")
        axs[i_plot_bad_lfp].set_xlim([t_begin_plot, t_end_plot])
        axs[i_plot_bad_lfp].vlines([t_sz_begin_optical, t_sz_end_optical], ymin=np.min(lfp_y), ymax=np.max(lfp_y), color="green")  # optical seizure start, end
        i_plot_bad_lfp += 1

if save_fig:
    fpath_out = os.path.join(output_dir, "bad_lfp_tmev.pdf")
    plt.savefig(fpath_out, format="pdf", bbox_inches="tight")
    print(f"Saved to {fpath_out}")
plt.show()